---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: E-Commerce

### 📋 **Topic**: Pricing Behavior Analysis with Python

🚫 **Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---


## Overview

Let's use our Python knowledge to see how often providers in a two-sided market change their prices. This analysis will help us understand pricing dynamics and seller behavior in digital marketplaces.

**What we'll learn:**
- How to analyze pricing behavior over time
- How to calculate price change frequencies
- How to visualize distribution patterns
- Understanding seller behavior in digital markets


In [ ]:
# Let's import the libraries we will use
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Load the prices dataset
df_prices = pd.read_csv("../data/prices.csv")

print("Dataset loaded successfully!")
print(f"Dataset shape: {df_prices.shape}")
print(f"Columns: {df_prices.columns.tolist()}")

# This is price data simulated from a real marketplace
# We want to figure out how often providers change their prices!

print("Sample of price data:")
print(df_prices.head(10))

print(f"Time range: {df_prices['time_m'].min()} to {df_prices['time_m'].max()}")
print(f"Number of unique cars: {df_prices['car'].nunique()}")
print(f"Total observations: {len(df_prices)}")


In [ ]:
# Calculate price changes per car
# For each car, we want to know how many times they changed their price
# relative to how long they were in the dataset

df_changes = (
    df_prices.groupby("car").agg({"time_m": ["count", "min", "max"]}).reset_index()
)

# Flatten column names
df_changes.columns = ["car", "n", "start_t", "end_t"]

# Calculate price changes rate
df_changes["price_changes"] = df_changes["n"] / (df_changes["end_t"] - df_changes["start_t"] + 1)

print("Sample of price changes data:")
print(df_changes.head())

print(f"Average observations per car: {df_changes['n'].mean():.2f}")
print(f"Average price changes per period: {df_changes['price_changes'].mean():.3f}")


---

## 🎉 Summary

We analyzed pricing behavior in an online marketplace and found:
- **Price change frequency** analysis by seller
- **Distribution patterns** of pricing behavior
- **Seller behavior insights** in digital markets

The data reveals how often different sellers adjust their prices, providing insights into market dynamics.

### Next:
We'll learn about randomization and experimental design

---
